In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import pickle
with open('Paruosti_duomenys.pkl', 'rb') as file:
    df_Elektra = pickle.load(file)

In [5]:
def KNN_regresorius(df, skaicius=5):
    df_test = df
    X = df_test.drop(columns=['Viena laiko zona'])
    y = df_test['Viena laiko zona']

    # Padalijame į treniravimo ir testavimo rinkinius
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Sukuriame polinominius požymius
    # poly = PolynomialFeatures(degree=2, include_bias=False)
    # X_train_poly = poly.fit_transform(X_train)
    # X_test_poly = poly.transform(X_test)

    # Normalizuojame duomenis
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Sukuriame K-nn Regressor
    knn = KNeighborsRegressor()

    # Nustatome hiperparametrų tinklą
    param_grid = {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan','minkowski']
    }

    # Naudojame GridSearchCV geriausiems parametrams rasti
    grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=skaicius, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    # Geriausi parametrai
    best_params = grid_search.best_params_

    # Geriausias modelis
    best_model = grid_search.best_estimator_
    return best_model


In [6]:
model = KNN_regresorius(df_Elektra)

In [7]:
model

KNeighborsRegressor(metric='manhattan', n_neighbors=3, weights='distance')

In [8]:
# Išsaugome modelį
joblib.dump(model, 'regression_model.pkl')

['regression_model.pkl']

In [9]:
df_Elektra.head()

,Dieninė dedamoji,Naktinė dedamoji,Pastovioji dedamoji. EUR/mėn.,Viena laiko zona,Nord pool kaina,Metinė infliacija,Vidutinė Temperatūra C,Pagaminta elektra GWh
Data,,,,,,,,
2015-01,0.115,0.083,0.0,0.105,0.03978,-1.3,-0.38,0.357
2015-02,0.115,0.083,0.0,0.105,0.03944,-0.3,0.09,0.358
2015-03,0.115,0.083,0.0,0.105,0.03222,0.8,4.16,0.365
2015-04,0.115,0.083,0.0,0.105,0.03561,0.7,6.67,0.380
2015-05,0.115,0.083,0.0,0.105,0.03736,0.5,11.06,0.380


In [10]:
# Funkcija prognozei gauti pagal vartotojo įvestis
def spejimo_funkcija(D_Dedamoji, N_Dedamoji, Past_dedamoji, NordP_kaina, Infl, vid_temp, Pag_el):
    user_data = pd.DataFrame({
        'Dieninė dedamoji': [D_Dedamoji],
        'Naktinė dedamoji': [N_Dedamoji],
        'Pastovioji dedamoji. EUR/mėn.': [Past_dedamoji],
        'Nord pool kaina': [NordP_kaina],
        'Metinė infliacija': [Infl],
        'Vidutinė temperatūra C': [vid_temp],
        'Pagaminta elektra GWh': [Pag_el],
    })
    duomenu_spejimas = model.predict(user_data)
    return duomenu_spejimas[0]

# Pavyzdys, kaip vartotojas įveda duomenis

D_Dedamoji = float(input("Įveskite dieninės dedamosios vertę (rėžis tarp: 0.05 ir 0.5): "))
print(f"Dieninė dedamoji vertė: {D_Dedamoji}")

N_Dedamoji = float(input("Įveskite naktinės dedamosios vertę (rėžis tarp: 0.03 ir 0.35): "))
print(f"Naktinė dedamoji vertė: {N_Dedamoji}")

Past_dedamoji = int(input("Įveskite pastoviąją dedamąją (rėžis tarp 0 ir 5): "))
print(f"Pastovioji dedamoji: {Past_dedamoji}")

NordP_kaina = float(input("Įveskite Nord pool kainą (rėžis tarp 0.01 ir 0.4): "))
print(f"Nord pool kaina: {NordP_kaina}")

Infl = float(input("Įveskite infliacijos procentinę vertę (rėžis tarp -3 ir 15): "))
print(f"Infliacijos procentinė vertė: {Infl}")

vid_temp = float(input("Įveskite vidutinę temperatūrą (rėžis tarp -10 ir 25): "))
print(f"Vidutinė temperatūra: {vid_temp}")

Pag_el = float(input("Įveskite pagamintos el. kiekį (rėžis tarp: 0.2 ir 0.9): "))
print(f"Pagamintos el. kiekis: {Pag_el}")


kainos_spejimas = spejimo_funkcija(D_Dedamoji, N_Dedamoji, Past_dedamoji, NordP_kaina, Infl, vid_temp, Pag_el)
print("Prognozuojama kaina:", kainos_spejimas)

# except ValueError:
#     print("Įvesta neteisinga reikšmė. Bandykite dar kartą.")

Dieninė dedamoji vertė: 0.18
Naktinė dedamoji vertė: 0.13
Pastovioji dedamoji: 3
Nord pool kaina: 0.09
Infliacijos procentinė vertė: 2.5
Vidutinė temperatūra: 15.0
Pagamintos el. kiekis: 0.7
Prognozuojama kaina: 0.14464038862614054


C:\Users\Orion\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
